In [1]:
import pandas as pd
import sqlite3

In [ ]:
# make them data frames

df_emp = pd.read_csv('../data/raw/ssamatab1.csv', skiprows=2).dropna()
df_house = pd.read_csv('../data/raw/hpi_po_metro.csv')

In [3]:
# rename columns

df_emp.rename(columns={
    "Area FIPS Code" : "FIPS Code",
    "ST FIPS Code" : "State FIPS Code",
}, inplace=True)

In [4]:
df_emp.dtypes

LAUS Code                object
State FIPS Code         float64
FIPS Code               float64
Area                     object
Year                    float64
Month                   float64
Civilian Labor Force     object
Employment               object
Unemployment             object
Unemployment Rate        object
dtype: object

In [5]:
# type cast columns with floats to int64
 
df_emp = df_emp.astype({
    "Year" : "Int64",
    "Month" : "Int64",
    "State FIPS Code" : "Int64",
    "FIPS Code" : "Int64",
    })

In [6]:
# I am unable to type cast the object columns to in64. One possible reason could be there are empty strings. .isna() doesn't count strings, so check if there are any records with empty strings in object type columns.

(df_emp[['Employment', 'Unemployment', "Unemployment Rate", "Area", "LAUS Code", "Civilian Labor Force"]].values == '').sum()

np.int64(0)

In [7]:
object_columns = ['Employment', 'Unemployment', "Unemployment Rate", "Civilian Labor Force"]

In [8]:
# found a value called (n) which is a null value.
df_emp['Employment'].iloc[250:270]

251     45,222
252     28,974
253    314,110
254    182,327
255    900,488
256        (n)
257    747,794
258    384,580
259     48,122
260     58,004
261     58,093
262     66,958
263    286,826
264     56,608
265    212,807
266     25,634
267     42,227
268    476,049
269    374,891
270     41,323
Name: Employment, dtype: object

In [9]:
# 512 fields of (n). Need to replace or drop these somehow.
(df_emp[['Employment', 'Unemployment', "Unemployment Rate", "Area", "LAUS Code", "Civilian Labor Force"]].values == '(n)').sum()

np.int64(512)

In [10]:
df_emp.columns

Index(['LAUS Code', 'State FIPS Code', 'FIPS Code', 'Area', 'Year', 'Month',
       'Civilian Labor Force', 'Employment', 'Unemployment',
       'Unemployment Rate'],
      dtype='object')

In [11]:
# find rows with '(n)' and remove them from the dataset. 

mask = df_emp[object_columns].apply(lambda x: x.str.contains(r'\(n\)')).any(axis=1)

df_emp = df_emp[~mask]

In [12]:
for column in object_columns:
    df_emp[column] = df_emp[column].str.replace(",", "")
    df_emp[column] = df_emp[column].str.replace(".", "")
df_emp

,LAUS Code,State FIPS Code,FIPS Code,Area,Year,Month,Civilian Labor Force,Employment,Unemployment,Unemployment Rate
1,MT0111500000000,1,11500,"Anniston-Oxford, AL MSA",1990,1,51501,47884,3617,70
2,MT0112220000000,1,12220,"Auburn-Opelika, AL MSA",1990,1,53578,49408,4170,78
3,MT0113820000000,1,13820,"Birmingham, AL MSA",1990,1,458343,434097,24246,53
4,MT0119300000000,1,19300,"Daphne-Fairhope-Foley, AL MSA",1990,1,46132,43694,2438,53
5,MT0119460000000,1,19460,"Decatur, AL MSA",1990,1,65284,60964,4320,66
...,...,...,...,...,...,...,...,...,...,...
168200,MT7211640000000,72,11640,"Arecibo, PR MSA",2025,8,68674,64402,4272,62
168201,MT7225020000000,72,25020,"Guayama, PR MSA",2025,8,19243,17562,1681,87
168202,MT7232420000000,72,32420,"Mayaguez, PR MSA",2025,8,68825,62866,5959,87
168203,MT7238660000000,72,38660,"Ponce, PR MSA",2025,8,90856,83087,7769,86


In [13]:
for column in object_columns:
    df_emp[object_columns].astype(int)

In [27]:
# create the database
connection = sqlite3.connect("../database/database.db")

# create the tables in the database

df_emp.to_sql("unemployment", connection, if_exists="replace", index=False)
df_house.to_sql("housing_value", connection, if_exists="replace", index=False)


13800